In [1]:
# Imports
# Gymnasium imports
import gymnasium as gym
from gymnasium import Env
from gymnasium.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete
import networkx as nx
from networkx.drawing.nx_agraph import graphviz_layout
# Import helpers
import numpy as np
import pandas as pd
import random
import os
import json
import csv
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import product
from pathlib import Path
import pickle
from datetime import datetime
import optuna
from collections import deque
# Import stable baselines
from stable_baselines3 import PPO, A2C
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv, VecNormalize
from stable_baselines3.common.env_util import make_vec_env, SubprocVecEnv
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_checker import check_env

In [2]:
class SS_Mngmt_Env(Env):
    """
    Supply Chain Management Environment
    Environment for supply chain management with a single product and multiple nodes.
    The action space constists of the order quantities for each node.
    The observation space consists of the inventory levels and the planned and actual demand for each node.
    """
    metadata = {"render_modes": ["human"], "render_fps": 4}
    # Define the action and observation space
    def __init__(
        self,
        EP_LENGTH=52,
        network_config=None,
        render_mode=None,
        model_type=None,
        stockout_cost=500,  # Cost of stockout
        stock_out_max=9,  # Maximum number of stockouts
        order_cost=5,  # Cost of each order
        item_cost=0.1,  # Cost of each item
        stock_cost=0.5,  # Cost of stock per unit
        item_prize=20,  # Prize of each item
        order_quantities=[0, 15, 50],  # Order quantities for each node
        demand_mean=10,  # Mean demand
        demand_std=2,  # Standard deviation of demand
        demand_noise=0,  # Mean noise in demand
        demand_noise_std=2,  # Standard deviation of noise in demand
        demand_prob=0.4,  # Probability of having demand
        intermediate_reward=1000,  # Intermediate reward
        progressive_stock_cost=False,  # Progressive stock cost
        kaggle=False,  # Kaggle mode (True or False)
        extreme=False,  # Extreme mode (True or False)
        seasonality=False,  # Seasonality mode (True or False)
    ):
        """
        Initialize the environment
        EP_LENGTH: int - Total length of the episode
        network_config: str - JSON string with network configuration
        render_mode: str - Render mode for the environment
        model_type: str - Type of model (e.g., PPO, A2C)
        stockout_cost: float - Cost of stockout
        stock_out_max: int - Maximum number of stockouts
        order_cost: float - Cost of each order
        item_cost: float - Cost of each item
        stock_cost: float - Cost of stock per unit
        item_prize: float - Prize of each item
        order_quantities: list - Order quantities for each node
        demand_mean: float - Mean demand
        demand_std: float - Standard deviation of demand
        demand_noise: float - Mean noise in demand
        demand_noise_std: float - Standard deviation of noise in demand
        demand_prob: float - Probability of having demand
        progressive_stock_cost: bool - Progressive stock cost
        kaggle: bool - Kaggle mode (True or False)
        """
        self.EP_LENGTH = EP_LENGTH  # Total length
        self.episode_length = EP_LENGTH  # Current length of the episode
        self.intermediate_reward = intermediate_reward  # Intermediate reward
        self.total_reward = 0
        self.model_type = model_type
        # Set the data path
        now = datetime.now()
        self.data_path = (
            f'./Data/{now.strftime("%Y-%m-%d")}_environment_data_{self.model_type}.csv'
        )
        # Set the fieldnames for the CSV file
        self.fieldnames = [
            "Time",
            "Node",
            "Stock",
            "Action",
            "Demand",
            "Delivery",
            "Reward",
            "Total Reward",
            "Backlog",
        ]
        self.file_initialized = False
        # Seting up the network
        self.network_config = network_config
        self.graph = nx.DiGraph()
        self.setup_network(self.network_config)
        self.lead_times = nx.get_edge_attributes(self.graph, "L")
        # Number of nodes excluding 'S' and 'D'
        num_nodes = len(self.graph.nodes) - 2
        # Define the costs
        self.stockout_cost = stockout_cost
        self.order_cost = order_cost
        self.item_cost = item_cost
        self.stock_cost = stock_cost
        self.stock_out_max = stock_out_max
        self.item_prize = item_prize
        self.progressive_stock_cost = progressive_stock_cost
        self.stock_out_counter = 0
        self.order_quantities = order_quantities
        # Order delay and queue
        self.order_queues = self.order_queue(initial_order=order_quantities[1])
        # Backlog queue for each node
        self.backlog_queues = self.backlog_queue()
        # Define action space
        n_actions = len(order_quantities)
        n_nodes = len(self.graph.nodes) - 2
        action_choices = np.full(n_nodes, n_actions)
        self.action_space = MultiDiscrete(action_choices)
        max_lead_time = max([data["L"] for _, _, data in self.graph.edges(data=True)])
        self.observation_space = Dict(
            {
                "inventory_levels": Box(
                    low=0, high=1000, shape=(num_nodes,), dtype=np.float32
                ),
                "current_demand": Box(
                    low=0, high=1000, shape=(num_nodes,), dtype=np.float32
                ),
                "backlog_levels": Box(
                    low=0, high=1000, shape=(num_nodes,), dtype=np.float32
                ),
                "order_queues": Box(
                    low=0, high=1000, shape=(num_nodes, max_lead_time), dtype=np.float32
                ),
                "lead_times": Box(
                    low=1, high=max_lead_time, shape=(num_nodes,), dtype=np.int32
                ),
            }
        )
        # Setting up the initial state
        self.demand_mean = demand_mean
        self.demand_std = demand_std
        self.demand_noise = demand_noise
        self.demand_noise_std = demand_noise_std
        self.demand_prob = demand_prob
        self.extreme = extreme
        self.seasonality = seasonality
        if self.seasonality == True:
            self.planned_demands = self.planned_demand_seasonality(
                self.demand_mean, self.demand_std, self.demand_prob
            )
        else:
            self.planned_demands = self.planned_demand(
                self.demand_mean, self.demand_std, self.demand_prob
            )
        if self.extreme == True:
            self.actual_demands = self.actual_demand_extremes(
                self.planned_demands, self.demand_noise_std, self.demand_noise
            )
        else:
            self.actual_demands = self.actual_demand(
                self.planned_demands, self.demand_noise_std, self.demand_noise
            )
        # Collect initial inventories from the graph
        initial_inventories = []
        for node in self.graph.nodes:
            if node not in ["S", "D"]:
                initial_inventories.append(self.graph.nodes[node].get("I", 0))
        initial_inventories = np.array(initial_inventories, dtype=np.float32).flatten()
        self.state = {
            "inventory_levels": initial_inventories.astype(np.float32),
            "planned_demand": self.planned_demands,
            "actual_demand": self.actual_demands,
            "current_demand": self.actual_demands[0],
            "backlog_levels": np.zeros(num_nodes),
            "order_queue_status": np.zeros(num_nodes),
        }
        # Prep to save the data
        self.inventory = initial_inventories
        self.stock_history = [self.inventory.tolist()]
        self.reward_history = [np.sum(initial_inventories * self.stock_cost * -1)]
        # Kaggle mode
        self.kaggle = kaggle
        # Render mode
        self.render_mode = render_mode
        self.screen_initialized = False
    # Defining the step function
    def step(self, action):
        """
        Executes one step in the environment.
        Starts by processing the orders and updating the inventory levels for each node.
        Then, it computes the reward based on the order costs and stock level.
        Finally, it checks if the episode is done and returns the next state, reward, and whether the episode is done.
        """
        # Returns the next state, reward and whether the episode is done
        timestep = self.EP_LENGTH - self.episode_length
        # num_nodes = len(self.graph.nodes) - 2
        # Retrieve the current inventory levels
        self.inventory = self.state["inventory_levels"]
        inventory_levels = np.copy(self.inventory)
        reward = 0
        # Retrieve the actual demand for the current timestep
        self.current_demand = self.actual_demands[timestep].astype(np.float32)
        # Add every first element of the order queues to the history
        self.new_order = [self.order_quantities[i] for i in action]
        # For visualization and history data
        self.orders = np.array(
            [
                self.order_queues[node][0]
                for node in self.graph.nodes
                if node not in ["S", "D"]
            ]
        )
        # Process the orders and update the inventory levels for each node
        for node in self.graph.nodes:
            if node not in ["S", "D"]:
                node_index = self.node_to_index(node)
                # Deduct costs for placing new orders
                if self.new_order[node_index] > 0:
                    reward -= self.order_cost + (
                        self.new_order[node_index] * self.item_cost
                    )
                # Fulfill orders from the queue
                order = self.order_queues[node].popleft()
                inventory_levels[node_index] += order
                # Attempt to meet current demand
                node_demand = self.current_demand[node_index]
                if inventory_levels[node_index] >= node_demand:
                    # Enough stock to meet demand
                    inventory_levels[node_index] -= node_demand
                    reward += node_demand * self.item_prize
                else:
                    # Insufficient stock - add unmet demand to backlog and apply penalty
                    unmet_demand = node_demand - inventory_levels[node_index]
                    inventory_levels[node_index] -= node_demand - unmet_demand
                    reward += (node_demand - unmet_demand) * self.item_prize
                    reward -= self.stockout_cost * unmet_demand  # Apply stockout cost
                    self.backlog_queues[node].append(unmet_demand)
                    # Increment the stockout counter
                    self.stock_out_counter += 1
                # Process backlog with any remaining stock
                while self.backlog_queues[node] and inventory_levels[node_index] > 0:
                    backlog_demand = self.backlog_queues[node][0]
                    if inventory_levels[node_index] >= backlog_demand:
                        inventory_levels[node_index] -= backlog_demand
                        reward += backlog_demand * self.item_prize
                        self.backlog_queues[node].popleft()
                    else:
                        break  # Not enough stock to clear the backlog completely
                # backlog penalty
                reward -= self.stockout_cost * len(self.backlog_queues[node])
                # Replenish order queue
                self.order_queues[node].append(self.new_order[node_index])
        if self.progressive_stock_cost == False:
            # Compute the reward based on the order costs and stock level
            reward -= np.sum(inventory_levels) * self.stock_cost
        elif self.progressive_stock_cost == True:
            reward -= np.sum(
                [
                    self.quadratic_stock_cost(self.stock_cost, inv)
                    for inv in inventory_levels
                ]
            )
        # Penalty if the episode cannot be completed
        if self.stock_out_counter >= self.stock_out_max:
            reward -= (
                self.episode_length * self.stockout_cost * (len(self.graph.nodes) - 2)
            )
        # Intermediate reward
        if timestep % 2 == 0:
            reward += self.intermediate_reward
        # Update the reward
        self.total_reward += reward
        # Decrease the episode length
        self.episode_length -= 1
        inventory_levels = inventory_levels.flatten()
        self.inventory = inventory_levels
        self.state = {
            "inventory_levels": inventory_levels.astype(np.float32),
            "planned_demand": self.planned_demands,
            "actual_demand": self.actual_demands,
            "current_demand": self.actual_demands[timestep],
            "backlog_levels": self.backlog_queues,
            "order_queue_status": self.order_queues,
        }
        max_lead_time = max([data["L"] for _, _, data in self.graph.edges(data=True)])
        obs = {
            "inventory_levels": self.inventory.astype(np.float32),
            "current_demand": self.actual_demands[timestep].astype(np.float32),
            "backlog_levels": np.array(
                [len(queue) for queue in self.backlog_queues.values()], dtype=np.float32
            ),
            "order_queues": np.array(
                [
                    list(self.order_queues[node])
                    + [0] * (max_lead_time - len(self.order_queues[node]))
                    for node in self.graph.nodes
                    if node not in ["S", "D"]
                ],
                dtype=np.float32,
            ),
            "lead_times": np.array(
                [
                    len(self.order_queues[node])
                    for node in self.graph.nodes
                    if node not in ["S", "D"]
                ],
                dtype=np.int32,
            ),
        }
        # Update the history data
        self.reward_history.append(reward)
        self.stock_history.append(list(self.inventory))
        # self.log_step_data(timestep, action, reward)
        # Check if the episode is done
        done = self.episode_length == 0
        # Check if episode is done
        if self.stock_out_counter >= self.stock_out_max:
            done = True
        elif self.episode_length <= 0:
            done = True
        else:
            done = False
        # Set placeholder for info
        info = {}
        # Check if the episode is truncated
        truncated = False
        return obs, float(reward), done, truncated, info
    def quadratic_stock_cost(self, stock_cost, inventory_level):
        """
        Quadratic stock cost function.
        """
        return stock_cost * (inventory_level**2)  # Quadratic cost
    def log_step_data(self, timestep, action, reward):
        if not self.file_initialized:
            with open(self.data_path, "w", newline="") as csvfile:
                writer = csv.DictWriter(csvfile, fieldnames=self.fieldnames)
                writer.writeheader()
            self.file_initialized = True  # Mark as initialized
        for n in range(len(self.inventory)):
            node_name = self.get_node_name(n)
            row = {
                "Time": timestep + 1,
                "Node": node_name,
                "Stock": self.inventory[n],
                "Action": self.new_order[n],
                "Demand": self.current_demand[n],
                "Delivery": self.orders[n],
                "Reward": reward,
                "Total Reward": self.total_reward,
                "Backlog": len(self.backlog_queues[node_name]) > 0,
            }
            # Append the row to the CSV file
            with open(self.data_path, "a", newline="") as csvfile:
                writer = csv.DictWriter(csvfile, fieldnames=self.fieldnames)
                writer.writerow(row)
    def reward_function(self):
        # TODO - Implement a custom reward function
        return 0
    def render(self):
        # Just check episode lenghth and only plot the last one when using matplotlib
        if self.render_mode is not None:
            if self.render_mode == "human":
                self.render_human()
    def render_human(self):
        """
        Renders the environment in human mode.
        Useful for debugging and visualization.
        """
        print("*" * 50)
        print("\nEpisode Information")
        print(f"Episode Length: {self.EP_LENGTH - self.episode_length}")
        if len(self.stock_history) > 1:
            print(f"Stock Level (Previous Timestep): {self.stock_history[-2]}")
        else:
            print(
                "Stock Level (Previous Timestep): No previous timestep data available"
            )
        print(f"Stock Level: {self.inventory}")
        print(
            f"Planned Demand: {self.planned_demands[self.EP_LENGTH - self.episode_length - 1]}"
        )
        print(f"Actual Demand: {self.current_demand}")
        print(f"Action: {self.new_order}")
        print(f"Deliveries: {self.orders}")
        # print(
        #     f"Previous Reward: {self.reward_history[self.EP_LENGTH - self.episode_length - 1]}"
        # )
        print(f"Step Reward: {self.reward_history[-1]}")
        print(f"Total Reward: {self.total_reward}")
        print("\nBacklog:")
        print([len(queue) > 0 for queue in self.backlog_queues.values()])
        # pprint(self.backlog_queues, indent=4)
        print("\nOrder Queue:")
        pprint(self.order_queues, indent=4)
        print()
        # print("Stockout Cost: ", self.stockout_cost)
        print("\nStockout Counter: ", self.stock_out_counter)
        return
    def setup_network(self, network_config=None):
        """
        Sets up the network graph based on the configuration provided.
        """
        config = json.loads(network_config)
        # Add nodes to the graph
        for node, attributes in config["nodes"].items():
            self.graph.add_node(node, **attributes)
        # Add edges to the graph with lead times
        for edge in config["edges"]:
            self.graph.add_edge(edge["source"], edge["target"], L=edge["L"])
    def render_network(self):
        """
        Renders the network graph using NetworkX and Matplotlib.
        """
        print("Node Attributes:")
        for node, attributes in self.graph.nodes(data=True):
            print(f"Node {node}: {attributes}")
        pos = graphviz_layout(self.graph, prog="dot")
        plt.figure(figsize=(8, 6))
        nx.draw_networkx_nodes(self.graph, pos, node_size=700, node_color="lightblue")
        nx.draw_networkx_edges(
            self.graph, pos, edgelist=self.graph.edges(), arrowstyle="->", arrowsize=20
        )
        nx.draw_networkx_labels(self.graph, pos, font_size=12, font_family="sans-serif")
        edge_labels = nx.get_edge_attributes(self.graph, "L")
        nx.draw_networkx_edge_labels(self.graph, pos, edge_labels=edge_labels)
        plt.title("Supply Chain Network Graph", fontsize=15)
        # Display the plot
        plt.axis("off")
        plt.show()
    def node_to_index(self, node):
        """
        Returns the index of the node given its name.
        """
        return list(self.graph.nodes).index(node)
    def get_node_name(self, index):
        """
        Returns the name of the node given its index.
        """
        return list(self.graph.nodes)[index]
    def planned_demand(self, demand_mean=10, demand_std=2, demand_prob=0.8):
        """
        Generates planned demand for each edge in the network over the whole episode.
        """
        edges_leading_to_D = [edge for edge in self.graph.edges if edge[1] == "D"]
        planned_demand = np.zeros((self.EP_LENGTH, len(edges_leading_to_D)))
        for i, edge in enumerate(edges_leading_to_D):
            for j in range(self.EP_LENGTH):
                # Introduce a probability of having demand
                if np.random.rand() < demand_prob:
                    planned_demand[j, i] = int(
                        np.random.normal(demand_mean, demand_std)
                    )
        return planned_demand
    import numpy as np
    def planned_demand_seasonality(
        self,
        demand_mean=10,
        demand_std=2,
        demand_prob=0.8,
        season_length=50,
        seasonality_strength=0.8,
    ):
        """
        Generates planned demand for each edge in the network over the whole episode,
        incorporating seasonality into the demand pattern.
        Parameters:
        - demand_mean: Mean of the demand.
        - demand_std: Standard deviation of the demand.
        - demand_prob: Probability of demand occurring.
        - season_length: Length of the seasonal cycle.
        Returns:
        - planned_demand: 2D array of planned demand with seasonality.
        """
        edges_leading_to_D = [edge for edge in self.graph.edges if edge[1] == "D"]
        planned_demand = np.zeros((self.EP_LENGTH, len(edges_leading_to_D)))
        # Generate seasonality factors for the episode length
        seasonality_factors = 1 + seasonality_strength * np.sin(
            2 * np.pi * np.arange(self.EP_LENGTH) / season_length
        )
        for i, edge in enumerate(edges_leading_to_D):
            for j in range(self.EP_LENGTH):
                # Introduce a probability of having demand
                if np.random.rand() < demand_prob:
                    # Apply seasonality to the mean demand
                    seasonal_mean = demand_mean * seasonality_factors[j]
                    planned_demand[j, i] = int(
                        np.random.normal(seasonal_mean, demand_std)
                    )
        return planned_demand
    def planned_demand_even(self, demand_mean=10, demand_std=2):
        """
        Generates planned demand for each edge in the network over the whole episode.
        The demand is distributed evenly, occurring only at fixed intervals (e.g., every fifth timestep).
        """
        # Get edges leading to "D"
        edges_leading_to_D = [edge for edge in self.graph.edges if edge[1] == "D"]
        # Initialize demand array with zeros
        planned_demand = np.zeros((self.EP_LENGTH, len(edges_leading_to_D)))
        for i, edge in enumerate(edges_leading_to_D):
            # Determine timesteps where demand occurs (e.g., every fifth timestep)
            timesteps_with_demand = np.arange(0, self.EP_LENGTH, 5)
            for j in timesteps_with_demand:
                # Generate demand from a normal distribution
                demand = max(
                    0, np.random.normal(demand_mean, demand_std)
                )  # Ensure non-negative demand
                planned_demand[j, i] = int(demand)
        return planned_demand
    def actual_demand(self, planned_demand, demand_noise_std=2, demand_noise=2):
        """
        Generates a random actual demand for each edge in the network based on the planned demand from the current timestep.
        """
        actual_demand = np.copy(planned_demand)
        for i in range(actual_demand.shape[0]):
            for j in range(actual_demand.shape[1]):
                # Add a small random noise to the planned demand
                if planned_demand[i, j] > 0:
                    noise = np.random.normal(demand_noise, demand_noise_std)
                    # Ensure actual demand is not less than 0
                    actual_demand[i, j] = int(max(0, actual_demand[i, j] + noise))
        return actual_demand
    def actual_demand_extremes(
        self,
        planned_demand,
        demand_noise_std=2,
        demand_noise=2,
        n_multiplications=[3, 6],
        size_multiplier=[3, 4],
    ):
        """
        Generates a random actual demand for each edge in the network based on the planned demand from the current timestep.
        To create a more interesting scenario, the demand is multiplied by a random factor 3-5 times over the whole demand.
        """
        actual_demand = np.copy(planned_demand)
        # Add random noise to planned demand
        for i in range(actual_demand.shape[0]):
            for j in range(actual_demand.shape[1]):
                if planned_demand[i, j] > 0:
                    noise = np.random.normal(demand_noise, demand_noise_std)
                    actual_demand[i, j] = int(max(0, actual_demand[i, j] + noise))
        # Multiply demand by a random factor 3-5 times
        num_multiplications = np.random.randint(
            n_multiplications[0], n_multiplications[1]
        )  # Randomly select between 3 and 5
        for _ in range(num_multiplications):
            # Select random indices
            i = np.random.randint(0, actual_demand.shape[0])
            j = np.random.randint(0, actual_demand.shape[1])
            # Apply a random multiplier
            if planned_demand[i, j] > 0:
                multiplier = np.random.uniform(size_multiplier[0], size_multiplier[1])
                actual_demand[i, j] = int(actual_demand[i, j] * multiplier)
        return actual_demand
    def order_queue(self, initial_order=10):
        """
        Creates a dictionary for the order queues for each node in the network.
        """
        order_queues = {}
        for node in self.graph.nodes:
            if node not in ["S", "D"]:
                in_edges = list(self.graph.in_edges(node, data=True))
                if in_edges:
                    lead_time = in_edges[0][2]["L"]
                    order_queues[node] = deque(
                        [initial_order] + [0] * (lead_time - 1), maxlen=lead_time
                    )
        return order_queues
    def backlog_queue(self):
        """
        Creates a dictionary for the backlog queues for each node in the network.
        """
        backlog_queues = {}
        for node in self.graph.nodes:
            if node not in ["S", "D"]:
                in_edges = list(self.graph.in_edges(node, data=True))
                if in_edges:
                    backlog_queues[node] = deque()
        return backlog_queues
    def save_state(self):
        """
        Saves the current state of the environment.
        Used for greedy algorithm.
        """
        return {
            "episode_length": self.episode_length,
            "inventory": np.copy(self.inventory),
            "total_reward": self.total_reward,
            "state": self.state,
            "order_queues": {k: deque(v) for k, v in self.order_queues.items()},
            "backlog_queues": {k: deque(v) for k, v in self.backlog_queues.items()},
        }
    def load_state(self, saved_state):
        """
        Loads the state of the environment.
        Used for greedy algorithm.
        """
        self.episode_length = saved_state["episode_length"]
        self.inventory = saved_state["inventory"]
        self.total_reward = saved_state["total_reward"]
        self.state = saved_state["state"]
        self.order_queues = {
            k: deque(v) for k, v in saved_state["order_queues"].items()
        }
        self.backlog_queues = {
            k: deque(v) for k, v in saved_state["backlog_queues"].items()
        }
    def reset(self, seed=None, options=None):
        """
        Resets the environment to the initial state.
        """
        super().reset(seed=seed)  # Reset the seed
        if seed is not None:
            random.seed(seed)
        # Reset the episode length
        self.episode_length = self.EP_LENGTH
        self.file_initialized = False
        self.total_reward = 0
        # Reset the network
        self.graph = nx.DiGraph()
        self.setup_network(self.network_config)
        num_nodes = len(self.graph.nodes) - 2
        # Order delay and backlog queue
        self.order_queues = self.order_queue(initial_order=self.order_quantities[1])
        self.backlog_queues = self.backlog_queue()
        self.stock_out_counter = 0
        # Define the initial state
        if self.seasonality == True:
            self.planned_demands = self.planned_demand_seasonality(
                self.demand_mean, self.demand_std, self.demand_prob
            )
        else:
            self.planned_demands = self.planned_demand(
                self.demand_mean, self.demand_std, self.demand_prob
            )
        if self.extreme == True:
            self.actual_demands = self.actual_demand_extremes(
                self.planned_demands, self.demand_noise_std, self.demand_noise
            ).astype(np.float32)
        else:
            self.actual_demands = self.actual_demand(
                self.planned_demands, self.demand_noise_std, self.demand_noise
            ).astype(np.float32)
        self.current_demand = self.actual_demands[0].astype(np.float32)
        # Collect initial inventories from the graph
        initial_inventories = []
        for node in self.graph.nodes:
            if node not in ["S", "D"]:
                initial_inventories.append(self.graph.nodes[node].get("I", 0))
        # Convert to numpy array
        initial_inventories = np.array(initial_inventories, dtype=np.float32).flatten()
        self.state = {
            "inventory_levels": initial_inventories,
            "planned_demand": self.planned_demands,
            "actual_demand": self.current_demand,
        }
        max_lead_time = max([data["L"] for _, _, data in self.graph.edges(data=True)])
        obs = {
            "inventory_levels": self.inventory.astype(np.float32),
            "current_demand": self.actual_demands[0].astype(np.float32),
            "backlog_levels": np.array(
                [len(queue) for queue in self.backlog_queues.values()], dtype=np.float32
            ),
            "order_queues": np.array(
                [
                    list(self.order_queues[node])
                    + [0] * (max_lead_time - len(self.order_queues[node]))
                    for node in self.graph.nodes
                    if node not in ["S", "D"]
                ],
                dtype=np.float32,
            ),
            "lead_times": np.array(
                [
                    len(self.order_queues[node])
                    for node in self.graph.nodes
                    if node not in ["S", "D"]
                ],
                dtype=np.int32,
            ),
        }
        # Resetting history data
        self.inventory = initial_inventories
        self.stock_history = [self.inventory.tolist()]
        self.reward_history = [np.sum(initial_inventories * self.stock_cost * -1)]
        # Placeholder for info
        info = {}
        return obs, info
class MultiDiscreteToBoxWrapper(gym.ActionWrapper):
    """
    Converts a MultiDiscrete action space to a Box action space.
    """
    def __init__(self, env):
        super().__init__(env)
        assert isinstance(
            env.action_space, MultiDiscrete
        ), "Environment must have a MultiDiscrete action space."
        self.original_action_space = env.action_space
        self.action_space = Box(
            low=0.0,
            high=1.0,
            shape=self.original_action_space.nvec.shape,
            dtype=np.float32,
        )
    def action(self, action):
        # Convert continuous action (Box) back to discrete (MultiDiscrete)
        discrete_action = np.round(
            action * (self.original_action_space.nvec - 1)
        ).astype(int)
        return discrete_action
    def reverse_action(self, action):
        # Optionally: Map discrete actions back to normalized (Box)
        normalized_action = action / (self.original_action_space.nvec - 1)
        return normalized_action


In [3]:
# Configuration of the network
with open("/kaggle/input/config-v0/network_config_v1.json") as file:
    network_config = file.read()
EP_LENGTH = 100  # Length of the episode

In [4]:
def load_config(config_file):
    with open(config_file, "r") as f:
        config = json.load(f)
    return config
def make_env(config_file="config.json"):
    config = load_config(config_file)
    env = SS_Mngmt_Env(
        network_config=network_config,
        EP_LENGTH=EP_LENGTH,
        render_mode="human",
        model_type="PPO",
        stockout_cost=config["stockout_cost"],
        order_cost=config["order_cost"],
        item_cost=config["item_cost"],
        stock_cost=config["stock_cost"],
        item_prize=config["item_prize"],
        progressive_stock_cost=config["progressive_stock_cost"],
        stock_out_max=config["stock_out_max"],
        order_quantities=config["order_quantities"],
        demand_mean=config["demand_mean"],
        demand_std=config["demand_std"],
        demand_noise=config["demand_noise"],
        demand_noise_std=config["demand_noise_std"],
        demand_prob=config["demand_prob"],
        extreme = True,
        seasonality = True,
    )
    return Monitor(env)
env = make_env("/kaggle/input/config-v0/env_config_v0.json")
check_env(env, warn=True)

/opt/conda/lib/python3.10/site-packages/stable_baselines3/common/env_checker.py:244: UserWarning: Your observation order_queues has an unconventional shape (neither an image, nor a 1D vector). We recommend you to flatten the observation to have only a 1D vector or use a custom policy to properly process the data.
  warnings.warn(


In [5]:
num_envs = 4
env_config_path = "/kaggle/input/config-v0/env_config_v0.json"
vec_env = SubprocVecEnv([lambda: make_env(env_config_path) for _ in range(num_envs)])
vec_env = VecNormalize(vec_env, norm_obs=True, norm_reward=True, clip_obs=10.0)

In [6]:
def optimize_hyperparams(trial):
    # Tune hyperparameters
    lr = trial.suggest_categorical("learning_rate", [1e-4, 5e-4, 1e-3])
    gamma = trial.suggest_categorical("gamma", [0.95, 0.99])
    ent_coef = trial.suggest_categorical("ent_coef", [0.01, 0.05, 0.1])
    vf_coef = trial.suggest_categorical("vf_coef", [0.5, 0.8, 1.0])
    n_steps = trial.suggest_categorical("n_steps", [8, 16, 32, 64])
    max_grad_norm = trial.suggest_categorical("max_grad_norm", [0.5, 1.0])
    use_rms_prop = trial.suggest_categorical("use_rms_prop", [True, False])
    gae_lambda = trial.suggest_categorical("gae_lambda", [0.8, 0.9, 0.95, 0.99])
    # Create and train the model
    model = A2C(
        "MultiInputPolicy",
        vec_env,
        learning_rate=lr,
        gamma=gamma,
        ent_coef=ent_coef,
        vf_coef=vf_coef,
        n_steps=n_steps,
        max_grad_norm=max_grad_norm,
        gae_lambda=gae_lambda,
        use_rms_prop=use_rms_prop,
        verbose=0,
    )
    model.learn(total_timesteps=750_000)
    # Evaluate the model
    rewards, _ = evaluate_policy(model, env, n_eval_episodes=5, return_episode_rewards=True)
    return sum(rewards) / len(rewards)
# Perform the optimization
study = optuna.create_study(direction="maximize", pruner=optuna.pruners.MedianPruner())
study.optimize(optimize_hyperparams, n_trials=30)
print("Best parameters:", study.best_params)

[I 2024-12-16 22:37:47,805] A new study created in memory with name: no-name-e9c21dc0-9e0f-4a70-a362-18c8c19b7ec1
[I 2024-12-16 22:51:28,655] Trial 0 finished with value: -398196.126 and parameters: {'learning_rate': 0.001, 'gamma': 0.99, 'ent_coef': 0.05, 'vf_coef': 0.8, 'n_steps': 16, 'max_grad_norm': 1.0, 'use_rms_prop': True, 'gae_lambda': 0.9}. Best is trial 0 with value: -398196.126.
[I 2024-12-16 23:03:54,259] Trial 1 finished with value: -48922.797999999995 and parameters: {'learning_rate': 0.0005, 'gamma': 0.95, 'ent_coef': 0.05, 'vf_coef': 1.0, 'n_steps': 32, 'max_grad_norm': 1.0, 'use_rms_prop': True, 'gae_lambda': 0.99}. Best is trial 1 with value: -48922.797999999995.
[I 2024-12-16 23:18:14,464] Trial 2 finished with value: -11772222.096 and parameters: {'learning_rate': 0.0001, 'gamma': 0.99, 'ent_coef': 0.1, 'vf_coef': 0.8, 'n_steps': 8, 'max_grad_norm': 0.5, 'use_rms_prop': False, 'gae_lambda': 0.99}. Best is trial 1 with value: -48922.797999999995.
[I 2024-12-16 23:31:

Best parameters: {'learning_rate': 0.0005, 'gamma': 0.95, 'ent_coef': 0.05, 'vf_coef': 1.0, 'n_steps': 32, 'max_grad_norm': 1.0, 'use_rms_prop': True, 'gae_lambda': 0.99}


In [7]:
now = datetime.now()
# Save the study to a file
with open(
    f"/kaggle/working/A2C_optuna_study_{now.strftime('%Y-%m-%d_%H_%M')}.pkl", "wb"
) as f:
    pickle.dump(study, f)